<a href="https://colab.research.google.com/github/stewartroge/UAM-VTSS/blob/main/UAM_VariableTacticalSpatialSeparationSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UAM Variable Tactical Separation System

In [1]:
import random
import numpy as np
from math import cos, sin, atan2, pi

# UAM conflict Detection

In [3]:
# Conflict detection is necessary for UAM air traffic Management as matter of Flight and operation safety

def conflict_detection(ownship, target, D, H, T):
    sx = abs(ownship['position'][0] - target['position'][0])
    sy = abs(ownship['position'][1] - target['position'][1])
    sz = abs(ownship['position'][2] - target['position'][2])
    vx = ownship['velocity'][0] - target['velocity'][0]
    vy = ownship['velocity'][1] - target['velocity'][1]
    vz = ownship['velocity'][2] - target['velocity'][2]

    if vx ** 2 + vy ** 2 == 0:
        if sx ** 2 + sy ** 2 < D ** 2:
            if abs(sz) < H:
                return True
            elif vz * sz < 0 and -H < vz * (T * vz + sz):
                return True
    else:
        a = vx ** 2 + vy ** 2
        b = 2 * (sx * vx + sy * vy)
        c = sx ** 2 + sy ** 2 - D ** 2
        discriminant = b ** 2 - 4 * a * c
        if discriminant >= 0:
            tDin = (-b - np.sqrt(discriminant)) / (2 * a)
            tDout = (-b + np.sqrt(discriminant)) / (2 * a)
            if abs(sz) < H and tDout > 0 and tDin < T:
                return True
            tHin = (H - sz) / vz if vz != 0 else float('inf')
            tHout = (-H - sz) / vz if vz != 0 else float('inf')
            tin = max(tDin, tHin)
            tout = min(tDout, tHout)
            if tin < tout and tout > 0 and tin < T:
                return True
    return False


# Wind Effects

In [ ]:
# Wind affects the flight of UAM evTOLs thus for purposes of realism and add complexity,the effects of wind and gusts are incorporated into the

In [4]:
def apply_wind_effect(velocity, wind_speed, wind_direction):
    # Calculate wind components in the UAM's velocity direction
    wind_component = wind_speed * cos(wind_direction - atan2(velocity[1], velocity[0]))

    # Adjust velocity components with wind effect
    velocity[0] += wind_component * cos(wind_direction)
    velocity[1] += wind_component * sin(wind_direction)
    return velocity


Wind is modeled as a mathematical function of velocity, wind speed and direction and inherently randomized for better realism

# VTSS primary Function

This is central algorithm that processes the input parameters to give an output of required variables that determine airspace allocation

In [5]:
def VTSS_UAM_algorithm(airspace_volume, UAM_speeds, protection_radius, design_separation, num_corridors, num_layers,
                       max_altitude, T):
    # Calculate corridor width
    corridor_width = (airspace_volume / num_corridors) ** (1 / 3)

    # Calculate lateral separation
    lateral_separation = corridor_width / 2 + 230

    # Calculate vertical separation
    vertical_separation = (max_altitude - protection_radius) / (num_layers - 1) + 300
    vertical_separation = max(vertical_separation, protection_radius)

    # Calculate longitudinal separation
    longitudinal_separation = corridor_width + 560

    # Calculate maximum number of vehicles that can fit in the airspace volume
    max_vehicles = int(airspace_volume / (corridor_width * vertical_separation * longitudinal_separation))

    # Create list to store positions and velocities of UAM vehicles
    vehicle_list = []

    # Generate random initial positions and velocities for vehicles within corridors, layers, and longitudinal segments
    for i in range(num_corridors):
        for j in range(num_layers):
            for k in range(max_vehicles):
                x = i * corridor_width + random.random() * corridor_width
                y = j * vertical_separation + random.random() * vertical_separation
                z = k * longitudinal_separation + random.random() * longitudinal_separation

                initial_position = [x, y, z]
                initial_velocity = [random.uniform(0, UAM_speeds[j]), random.uniform(0, UAM_speeds[j]), 0]

                vehicle_list.append({
                    'position': initial_position,
                    'velocity': initial_velocity,
                    'corridor': i,
                    'layer': j,
                    'longitudinal': z
                })

    # Apply wind effect to UAMs' velocities
    for vehicle in vehicle_list:
        wind_speed = random.uniform(0, 5)  # Example wind speed range
        wind_direction = random.uniform(0, 2 * pi)  # Example wind direction range
        vehicle['velocity'] = apply_wind_effect(vehicle['velocity'], wind_speed, wind_direction)

    # Count the number of conflicts
    conflicts = []
    for i in range(len(vehicle_list)):
        for j in range(i + 1, len(vehicle_list)):
            if conflict_detection(vehicle_list[i], vehicle_list[j], design_separation, vertical_separation, T):
                conflicts.append((i, j))
    num_conflicts = len(conflicts)

    # Calculate conflict rate
    conflict_rate = num_conflicts / (max_vehicles * (max_vehicles - 1) / 2)

    # Calculate time needed to resolve conflicts using 3D distances and velocities
    time_to_resolve = np.max(
        [np.linalg.norm(np.array(vehicle_list[i]['position']) - np.array(vehicle_list[j]['position']))
         / np.linalg.norm(np.array(vehicle_list[i]['velocity']) - np.array(vehicle_list[j]['velocity']))
         for i, j in conflicts]) if conflicts else 0

    # Return the final values
    return vertical_separation, lateral_separation, longitudinal_separation, conflict_rate, time_to_resolve, max_vehicles


# Input Parameters for the VTSS Algorithm

In [6]:
#  input parameters
airspace_volume = 556800000000
UAM_speeds = [30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]  # velocities for each UAM in each layer
protection_radius = 26
CPA = 90
num_corridors = 14
num_layers = 10
max_altitude = 800
T = 300  # Lookahead time in seconds

# Run the algorithm
vertical_separation, lateral_separation, longitudinal_separation, conflict_rate, time_to_resolve, max_vehicles = VTSS_UAM_algorithm(
    airspace_volume,
    UAM_speeds,
    protection_radius,
    CPA,
    num_corridors,
    num_layers,
    max_altitude,
    T)


# Output Variables of the Model

In [7]:
print(f"Conflict Rate: {conflict_rate:.4f}")
print(f"Time to Resolve Conflicts: {time_to_resolve:.2f} seconds")
print(f"Lateral Separation: {lateral_separation:.2f} meters")
print(f"Vertical Separation: {vertical_separation:.2f} meters")
print(f"Longitudinal Separation: {longitudinal_separation:.2f} meters")
print(f"Maximum Number of Vehicles: {max_vehicles}")


Conflict Rate: 0.0721
Time to Resolve Conflicts: 313.73 seconds
Lateral Separation: 1936.71 meters
Vertical Separation: 386.00 meters
Longitudinal Separation: 3973.43 meters
Maximum Number of Vehicles: 106
